In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import itertools, functools, operator
from sklearn.preprocessing import PolynomialFeatures
import os



In [97]:
# WEEK 3

#theta includes theta0
#theta are input as a (n,) array, returns all margins, change output to np.min for overall margin
def margin(data, labels, theta):
    points_num  = len(labels[0])
    ones = np.array([np.ones(points_num)])
    data = np.concatenate((data, ones), axis=0)
    margins = np.zeros(points_num)
    data = data.T
    for i in range(points_num):
        margins[i] = labels[0][i]*(np.dot(theta,data[i]))/np.linalg.norm(theta[:-1])
    return (margins)


def cv(data):
    return np.array([data])

def R(data):
    points_num = len(data[0])
    ones = np.array([np.ones(points_num)])
    data = np.concatenate((data, ones), axis=0)
    data = data.T
    R_arr = np.zeros(points_num)
    for i in range(points_num):
        R_arr[i] = np.linalg.norm(data[i])
    return np.max(R_arr)

def perceptron_max_mistakes(data, labels, theta):
    return (R(data)/margin(data, labels, theta))**2

#prints 2d data for perceptron
def graph_2d(data, labels,theta):
    data = data.T
    x,y  = data[:,0],data[:,1]
    x1,y1 = np.array([]), np.array([])
    x2,y2 = np.array([]), np.array([])

    for i in range(len(x)):
        
        if labels[0,i] == 1:
            x1 = np.concatenate((x1,np.array([x[i]])))
            y1 = np.concatenate((y1,np.array([y[i]])))
        else:
            x2 = np.concatenate((x2,np.array([x[i]])))
            y2 = np.concatenate((y2,np.array([y[i]])))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x1, y1, c='b', marker='o')
    ax.scatter(x2, y2, c='r', marker='o')
    ax.plot()
    x = np.linspace(0, 1, 1000)
    ax.plot(x , -(theta[0][0]/theta[0][1])*x - (theta[0][2]/theta[0][1]))
    plt.show()
    return None

#this function takes an initial value for theta, for one that defaults to theta_init = 0 see mit_wee2.ipynb
def perceptron(data, labels, params = {}, hook = None):    
    # if T not in params, default to 100
    T = params.get('T', 100)
    # Your implementation here
    d, n = data.shape
    theta = np.zeros((d,1)) 
    
     
    misclassified = np.zeros((n,1))
    count = 0
    theta_0 = np.zeros(1)
    #print("d = {}, n = {}, theta shape = {}, theta_0 shape = {}".format(d,n,theta.shape,theta_0.shape))
  
    for t in range(T):     
      for i in range(n):
        y = labels[0,i]
        x = data[:,i]
        
        a = np.dot(x,theta)+theta_0
        
        if np.sign(y*a) <=0: # update the thetas
          theta[:,0] = theta[:,0]+ y*x
          theta_0 = theta_0 + y
          misclassified[i][0] += 1
          count += 1
        #print(theta, theta_0)
          
    #print("shape x = {}, y = {}, theta = {}, theta_0 = {}".format(x.shape,y.shape,theta.shape,theta_0.shape))
    #print( "times misclassified = ", misclassified )
    print("count = ", count)
    return (theta,np.array([theta_0]))
        
def one_hot(x,k):
    arr = np.zeros((k,1))
    arr[x-1] = 1
    return arr
          
def discrete_to_onehot(data, k):
    point_num = len(data[0])
    
    data_onehot = np.zeros((k,point_num))
    for i in range(point_num):
        data_onehot[data[0,i]-1,i] = 1
    return data_onehot
          
def signed_dist(x,th,th0):
    return (np.dot(th.T,x)+th0)/np.sqrt(np.dot(th.T,th))

def poly_features(data, degree):
    poly = PolynomialFeatures(degree)
    return poly.fit_transform(data)

def load_auto_data(path_data):
    """
    Returns a list of dict with keys:
    """
    numeric_fields = {'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
                      'acceleration', 'model_year', 'origin'}
    data = []
    with open(path_data) as f_data:
        for datum in csv.DictReader(f_data, delimiter='\t'):
            for field in list(datum.keys()):
                if field in numeric_fields and datum[field]:
                    datum[field] = float(datum[field])
            data.append(datum)
    return data

#WEEK 4
def f1(x):
    return float((2 * x + 3)**2)

def df1(x):
    return 2 * 2 * (2 * x + 3)


def gd(f, df, x0, step_size_fn, max_iter):
    x = x0
    iter = 0
    fs = np.zeros(max_iter, dtype = object)
    xs = np.zeros(max_iter, dtype = object)
    for i in range(max_iter):
        fs[i] = f(x)
        xs[i] = x
        x  = x - step_size_fn(i) * df(x)
        iter += 1
    return x, fs, xs

def num_grad(f, delta=0.001):
    def df(x):
        grad = np.zeros_like(x, dtype=object)
        delta_i = np.zeros_like(x, dtype=object)
        for i in range(len(x)):
            delta_i[i] += delta
            grad[i] = (f(x+delta_i) - f(x - delta_i)) / (2*delta)
            delta_i[i] -= delta
        return grad
    return df

def minimize(f, x0, step_size_fn, max_iter):
    df = num_grad(f)
    return (gd(f, df, x0, step_size_fn, max_iter))

def hinge(v):
    arr = [0,1-v]
    return max(arr)

def hinge_loss(x, y, theta, theta_0):
    return hinge(y*(np.dot(theta.T,x) + theta_0))

def svm_obj(x, y, theta, theta_0, lam):
    n = len(x[0])
    L = 0 
    for i in range(n):
        L += hinge_loss(x[:,i], y[0,i], theta, theta_0)/n
    L += lam * np.dot(theta.T,theta)
    return L[0]

# Returns the gradient of hinge(v) with respect to v.
def d_hinge(v):
    print(v)
    arr = np.zeros_like(v, dtype=object)
    for i in range(len(v[0])):
        print(v[0][i])
        if 1-v[0][i] > 0:
            arr[0][i] = -1
        else:
            arr[0][i] = 0
    return arr
        
def d_hinge_loss_th(x, y, theta, theta_0):
    return d_hinge(y*(np.dot(theta.T,x) + theta_0))*y*x

def d_hinge_loss_th0(x, y, theta, theta_0):
    return d_hinge(y*(np.dot(theta.T,x) + theta_0))*y

def d_svm_obj_th(x, y, th, th0, lam):
    L_sum = 0
    for i in range(len(x[0])):
        L_sum += -d_hinge_loss_th(x[:,i], y[0,i], th, th0)*d_hinge(y[0,i]*(np.dot(th.T,x[:,i]) + th0))
    L_sum = L_sum/len(x[0])
    L_sum += lam * 2 * th.T
    return L_sum.T

def d_svm_obj_th0(x, y, th, th0, lam):
    L_sum = 0
    for i in range(len(x[0])):
        L_sum += -d_hinge_loss_th0( x[:,i], y[0,i], th, th0 )  *  d_hinge( y[0,i]*(np.dot(th.T,x[:,i]) + th0 ))
    L_sum = L_sum/len(x[0])
    return L_sum

def svm_obj_grad(x,y, th, th0, lam):
    return np.vstack([d_svm_obj_th(x,y,th,th0,lam), d_svm_obj_th0(x,y,th,th0,lam)])

#broken
def batch_svm_min(data, labels, lam):
    def f(th):
        return svm_obj(data, labels, th[:-1][0], th[-1][0], lam)
    def df(th):
        return svm_obj_grad(data, labels, th[:-1][0], th[-1][0], lam)
    def svm_min_step_size_fn(i):
       return 2/(i+1)**0.5
    return gd(f, df, np.zeros((3,1)), svm_min_step_size_fn, 100)

# WEEK 5

# Write a function that returns the gradient of lin_reg(x, th, th0)
# with respect to th
def d_lin_reg_th(x, th, th0):
    return x
    
# Write a function that returns the gradient of square_loss(x, y, th, th0) with
# respect to th.  It should be a one-line expression that uses lin_reg and
# d_lin_reg_th.
def d_square_loss_th(x, y, th, th0):
    return 2*(np.dot(np.transpose(th),x)+th0-y)*d_lin_reg_th(x,th,th0)

# Write a function that returns the gradient of mean_square_loss(x, y, th, th0) with
# respect to th.  It should be a one-line expression that uses d_square_loss_th.
def d_mean_square_loss_th(x, y, th, th0):
    return np.array([np.mean(d_square_loss_th(x,y,th,th0), axis=0)]).T


def sgd(X, y, J, dJ, w0, step_size_fn, max_iter):
    w = w0
    fs = []
    ws = []
    n = len(y[0])


In [ ]:
    np.random.seed(0)
    for i in range(max_iter):
        r = np.random.randint(n)
        Xr = X[:,r:r+1]
        yr = y[:,r:r+1]
        prev_f = J(Xr, yr, w) 
        prev_grad = dJ(Xr, yr, w)
        fs.append(prev_f)
        ws.append(w)
        if i == max_iter - 1:
            return w, fs, ws
        step = step_size_fn(i)
        w = w - step * prev_grad

In [99]:
X = np.array([[1., 2., 3., 4.], [1., 1., 1., 1.]])
Y = np.array([[1., 2.2, 2.8, 4.1]])
th = np.array([[1.0],[0.05]])
th0 = np.array([[0.]])

print(d_mean_square_loss_th(X[:,0:1], Y[:,0:1], th, th0).tolist())

[[0.10000000000000009]]
